<a href="https://colab.research.google.com/github/fmuigai2036/Moringa-chatbot/blob/main/test_chatbotfi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install tflearn
import numpy
import tflearn
import tensorflow
import random
import json
import nltk
nltk.download('punkt')
from tensorflow.python.framework import ops
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f1b602a8b7fc4e1c748e88c81402581fb304613c25de39daab4f1addccdb663b
  Stored in directory: /root/.cache/pip/wheels/4a/d5/f8/9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import json
with open('/content/FAQ (1)') as file:

    data = json.load(file)
print (data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'accommodation', 'patterns': ['do you offer accomodation', 'are there boarding facilities?', 'do you have hostels ?'], 'responses': ['No, we currently don’t provide accommodation. However, there are a number of private hostels along Ngong road, which are close to the school.'], 'context_set': ''}, {'tag': 'remote', 'patterns': ['Do you offer remote learning courses?', 'Are there online classes?', 'Can i take the course online?', 'Can i take virtual classes?'], 'responses': ['Moringa School’s remote classes are done live and online; with daily sessions where learners ask

In [4]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [5]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [6]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [7]:
training = numpy.array(training)
output = numpy.array(output)

In [8]:
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)


Training Step: 5999  | total loss: 0.11051 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.11051 - acc: 0.9944 -- iter: 40/44
Training Step: 6000  | total loss: 0.10400 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.10400 - acc: 0.9949 -- iter: 44/44
--


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the quora (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the quora (type quit to stop)!
You: certification
Yes, you will get a digital certificate after the successful completion of the program. A physical certificate can be printed for you at your cost.
